# Paintings by Style: CLIP + UMAP Scatter

This notebook mirrors the Picasso embedding plot, but uses `data/paintings-by-style.csv`. Points are colored by `style`, and hover shows artist, title, and year.

In [ ]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from sklearn.preprocessing import MinMaxScaler
import umap
import plotly.express as px
from IPython.display import display
try:
    from tqdm import tqdm
except Exception:
    tqdm = lambda x, **k: x

_HERE = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
_ROOT = _HERE.parent
DATA_DIR = _ROOT / 'data' / 'paintings-by-style'
CSV_PATH = _ROOT / 'data' / 'paintings-by-style.csv'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

df = pd.read_csv(CSV_PATH)
# Normalize column names for easier access
df = df.rename(columns={
    'directory+filename': 'rel_path',
    'painting title': 'title',
    'year of painting': 'year'
})
# Derive artist from rel_path: style/artist/filename
def extract_artist(p: str) -> str:
    parts = str(p).split('/')
    return parts[1] if len(parts) >= 2 else ''

df['artist'] = df['rel_path'].astype(str).map(extract_artist)
df['image_path'] = df['rel_path'].apply(lambda p: str(DATA_DIR / p))
df['year'] = pd.to_numeric(df['year'], errors='coerce')
print(f'Loaded {len(df)} rows from {CSV_PATH.name}')
display(df.head())

In [ ]:
# Compute CLIP embeddings
clip_model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32').to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')
clip_model.eval()

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return clip_processor(images=image, return_tensors='pt')['pixel_values']

embeddings = []
failed = []
for p in tqdm(df['image_path'], desc='Extracting CLIP embeddings'):
    try:
        pixel_values = preprocess_image(p).to(DEVICE)
        with torch.no_grad():
            emb = clip_model.get_image_features(pixel_values=pixel_values)
        embeddings.append(emb.squeeze().cpu().numpy())
    except Exception as e:
        failed.append((p, str(e)))
        embeddings.append(np.zeros(512, dtype=np.float32))

if failed:
    print(f'Failed to embed {len(failed)} images (filled with zeros).')

emb_matrix = np.vstack(embeddings)
reducer = umap.UMAP(random_state=42)
embedding_2d = reducer.fit_transform(emb_matrix)
df['x'] = embedding_2d[:, 0]
df['y'] = embedding_2d[:, 1]

In [ ]:
# Interactive scatter colored by style with informative hover
# Enforce equal axis range for a square canvas
xmin, xmax = df['x'].min(), df['x'].max()
ymin, ymax = df['y'].min(), df['y'].max()
xc, yc = (xmin + xmax) / 2.0, (ymin + ymax) / 2.0
half = max(xmax - xmin, ymax - ymin) / 2.0
half = 0.5 if half <= 0 else half

fig = px.scatter(
    df, x='x', y='y', color='style',
    hover_name='title',
    hover_data={'artist': True, 'year': True, 'style': False, 'rel_path': False},
    width=900, height=900,
)
fig.update_layout(
    title='CLIP Embedding of Paintings by Style (UMAP Projection)',
    template='plotly_white',
)
fig.update_xaxes(range=[xc - half, xc + half], visible=True, constrain='domain')
fig.update_yaxes(scaleanchor='x', scaleratio=1, visible=True, constrain='domain')
fig.show()